## Fundamentos del aprendizaje por refuerzo

El aprendizaje por refuerzo (RL, por sus siglas en inglés) es un paradigma del aprendizaje automático donde un agente aprende a tomar decisiones mediante la interacción con su entorno, recibiendo recompensas o penalizaciones según sus acciones. A diferencia del aprendizaje supervisado, donde el aprendizaje se basa en un conjunto de datos etiquetados, RL se basa en la retroalimentación recibida a partir de la experiencia directa. 

#### Elementos básicos del aprendizaje por refuerzo

En RL, los principales elementos son el agente, el entorno, los estados, las acciones y las recompensas. Estos elementos se relacionan de la siguiente manera:

1. **Agente**: Es el tomador de decisiones que interactúa con el entorno. El agente recibe información del entorno, toma decisiones (acciones) y aprende de las consecuencias de esas decisiones.

2. **Entorno**: Es el mundo con el que interactúa el agente. El entorno responde a las acciones del agente y proporciona nuevas observaciones y recompensas.

3. **Estados (s)**: Representan la situación actual del entorno. Un estado contiene toda la información necesaria para describir la situación en un momento dado.

4. **Acciones (a)**: Son las decisiones o movimientos que el agente puede realizar en un estado dado. El conjunto de todas las posibles acciones se denota como A.

5. **Recompensas (r)**: Son señales de retroalimentación que indican el valor de una acción en un estado específico. El objetivo del agente es maximizar la recompensa total a lo largo del tiempo.

La interacción entre estos elementos puede describirse como una serie de pasos, donde el agente percibe un estado del entorno, selecciona una acción, recibe una recompensa y transita a un nuevo estado.

#### Procesos de decisión de Markov (MDPs)

Los procesos de decisión de Markov (MDPs) son una herramienta matemática utilizada para modelar problemas de RL. Un MDP se define por los siguientes componentes:

1. **Conjunto de estados (S)**: Todos los posibles estados en los que el agente puede encontrarse.
2. **Conjunto de acciones (A)**: Todas las posibles acciones que el agente puede tomar.
3. **Función de transición (P)**: Describe la probabilidad de transición de un estado a otro, dado una acción. $P(s'|s,a)$ representa la probabilidad de moverse al estado $s'$ desde el estado $s$ tomando la acción $a$.
4. **Función de recompensa (R)**: Define la recompensa esperada al realizar una acción en un estado particular. $R(s,a)$ es la recompensa inmediata recibida al realizar la acción $a$ en el estado $s$.
5. **Factor de descuento (γ)**: Es un valor entre 0 y 1 que determina la importancia de las recompensas futuras. Un factor de descuento cercano a 0 hace que el agente se enfoque en recompensas inmediatas, mientras que un valor cercano a 1 da más peso a las recompensas futuras.

Los MDPs permiten formalizar el problema de RL y proporcionar una base para diseñar y analizar algoritmos.

#### Políticas y funciones de valor

En RL, una política ($\pi$) es una estrategia que el agente sigue para decidir qué acciones tomar en cada estado. Una política puede ser determinista ($\pi(s) = a$) o estocástica ($\pi(a|s)$), donde la acción es elegida con una cierta probabilidad.

Las funciones de valor son herramientas clave para evaluar la calidad de los estados y las acciones bajo una política determinada. Hay dos tipos principales de funciones de valor:

1. **Función de valor del estado (V)**: $V^\pi(s)$ es el valor esperado de las recompensas futuras comenzando desde el estado $s$ y siguiendo la política $\pi$. Se define como:
   $$
   V^\pi(s) = \mathbb{E}_\pi \left[ \sum_{t=0}^{\infty} \gamma^t r_{t+1} \mid s_t = s \right]
   $$

2. **Función de valor de la acción (Q)**: $Q^\pi(s,a)$ es el valor esperado de las recompensas futuras al tomar la acción $a$ en el estado $s$ y luego seguir la política $\pi$. Se define como:
   $$
   Q^\pi(s,a) = \mathbb{E}_\pi \left[ \sum_{t=0}^{\infty} \gamma^t r_{t+1} \mid s_t = s, a_t = a \right]
   $$

El objetivo del agente es encontrar la política óptima ($\pi^*$) que maximice estas funciones de valor.






#### Métodos basados en el valor

Los métodos basados en el valor se centran en aprender una función de valor que estima la calidad de los estados y acciones. Los dos algoritmos más representativos en esta categoría son Q-Learning y SARSA.

**Q-Learning**:
Q-Learning es un algoritmo off-policy que busca aprender la función de valor de acción $Q(s, a)$. La actualización de Q-Learning se basa en la ecuación de Bellman:

$$Q(s, a) \leftarrow Q(s, a) + \alpha [r + \gamma \max_{a'} Q(s', a') - Q(s, a)]$$

Aquí, $s$ es el estado actual, $a$ es la acción tomada, $r$ es la recompensa recibida, $s'$ es el nuevo estado, $\alpha$ es la tasa de aprendizaje, y $\gamma$ es el factor de descuento.


A continuación, presentamos un ejemplo de código en PyTorch para implementar Q-Learning, uno de los algoritmos de RL más conocidos, que aprende la función de valor de acción (Q).

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym
import warnings
warnings.filterwarnings("ignore")

# Definir la red neuronal para aproximar la función Q
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Función para elegir una acción basada en epsilon-greedy
def select_action(state, epsilon, q_network):
    if np.random.rand() < epsilon:
        return np.random.randint(action_dim)
    else:
        with torch.no_grad():
            state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
            q_values = q_network(state)
            return torch.argmax(q_values).item()

# Parámetros del entorno y de aprendizaje
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
lr = 0.001
gamma = 0.99
epsilon = 0.1
episodes = 500

q_network = QNetwork(state_dim, action_dim)
optimizer = optim.Adam(q_network.parameters(), lr=lr)
criterion = nn.MSELoss()

# Entrenamiento del agente
for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False
    while not done:
        action = select_action(state, epsilon, q_network)
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        
        # Actualización de la red Q
        next_state_tensor = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0)
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        action_tensor = torch.tensor([action], dtype=torch.int64)
        reward_tensor = torch.tensor([reward], dtype=torch.float32)
        done_tensor = torch.tensor([done], dtype=torch.float32)
        
        q_values = q_network(state_tensor)
        next_q_values = q_network(next_state_tensor)
        
        q_value = q_values.gather(1, action_tensor.unsqueeze(1)).squeeze(1)
        next_q_value = torch.max(next_q_values, 1)[0]
        expected_q_value = reward_tensor + gamma * next_q_value * (1 - done_tensor)
        
        loss = criterion(q_value, expected_q_value.detach())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        state = next_state
    
    print(f'Episodio {episode + 1}, Recompensa total: {total_reward}')

env.close()

**SARSA**:
    
SARSA (State-Action-Reward-State-Action) es un algoritmo on-policy que actualiza la función de valor de acción utilizando la acción actual y la siguiente acción seleccionada por la política. La actualización de SARSA se basa en la ecuación:

$$Q(s, a) \leftarrow Q(s, a) + \alpha [r + \gamma Q(s', a') - Q(s, a)]$$


#### Métodos basados en la política

Los métodos basados en la política se enfocan en aprender directamente una política que mapea estados a acciones, sin necesidad de una función de valor intermedia. Los dos algoritmos más comunes en esta categoría son REINFORCE y Actor-Critic.

**REINFORCE**:
REINFORCE es un algoritmo de gradiente de política que ajusta los parámetros de la política para maximizar la recompensa esperada. La actualización de la política se basa en la ecuación:

$$\theta \leftarrow \theta + \alpha \nabla_\theta \log \pi_\theta(a|s) G_t$$

Donde $G_t$ es la recompensa acumulada desde el tiempo $t$.


Presentamos una implementación de REINFORCE elemental.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym

class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.softmax(self.fc3(x), dim=-1)

def select_action(state, policy_network):
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
    action_probs = policy_network(state)
    action = torch.multinomial(action_probs, 1).item()
    return action, action_probs[0, action]

env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
lr = 0.001
gamma = 0.99
episodes = 500

policy_network = PolicyNetwork(state_dim, action_dim)
optimizer = optim.Adam(policy_network.parameters(), lr=lr)

for episode in range(episodes):
    state = env.reset()
    rewards = []
    log_probs = []
    total_reward = 0
    done = False
    while not done:
        action, log_prob = select_action(state, policy_network)
        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)
        log_probs.append(log_prob)
        total_reward += reward
        state = next_state
    
    discounted_rewards = []
    R = 0
    for r in rewards[::-1]:
        R = r + gamma * R
        discounted_rewards.insert(0, R)
    discounted_rewards = torch.tensor(discounted_rewards)
    
    policy_loss = 0
    for log_prob, reward in zip(log_probs, discounted_rewards):
        policy_loss += -log_prob * reward
    
    optimizer.zero_grad()
    policy_loss.backward()
    optimizer.step()
    
    print(f'Episodio {episode + 1}, Recompensa total: {total_reward}')

env.close()


**Actor-Critic**:
El método Actor-Critic combina una red de política (actor) y una red de valor (critic). El actor actualiza la política basándose en la ventaja estimada por el crítico. La actualización de la política sigue:
$$\theta \leftarrow \theta + \alpha \nabla_\theta \log \pi_\theta(a|s) A(s, a)$$

Y la actualización de la función de valor sigue:

$$\phi \leftarrow \phi + \beta \nabla_\phi (r + \gamma V_\phi(s') - V_\phi(s))$$

#### Métodos híbridos

Los métodos híbridos combinan enfoques basados en el valor y en la política para aprovechar las fortalezas de ambos. Entre los algoritmos híbridos más avanzados se encuentran DDPG, PPO y A3C.

**DDPG (Deep Deterministic Policy Gradient)**:
DDPG es un algoritmo off-policy que combina DQN y el actor-critic. Utiliza una red de actor para seleccionar acciones y una red crítico para evaluar la calidad de esas acciones. También emplea una red de destino para estabilizar el entrenamiento.

**PPO (Proximal Policy Optimization)**:
PPO es un algoritmo de política basada en la proximidad que busca mejorar la estabilidad del entrenamiento limitando el tamaño del paso de actualización. La función objetivo de PPO es:

$$ L^{CLIP}(\theta) = \mathbb{E}_t \left[ \min \left( \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)} A_t, \text{clip}\left( \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}, 1 - \epsilon, 1 + \epsilon \right) A_t \right) \right]$$


Realizamos una implementación de POO elemental.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.actor = nn.Linear(128, action_dim)
        self.critic = nn.Linear(128, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        policy_dist = torch.softmax(self.actor(x), dim=-1)
        value = self.critic(x)
        return policy_dist, value

def select_action(state, model):
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
    policy_dist, _ = model(state)
    action = torch.multinomial(policy_dist, 1).item()
    return action, torch.log(policy_dist[0, action])

env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
lr = 0.001
gamma = 0.99
epsilon = 0.2
episodes = 500

model = ActorCritic(state_dim, action_dim)
optimizer = optim.Adam(model.parameters(), lr=lr)

for episode in range(episodes):
    state = env.reset()
    rewards = []
    log_probs = []
    values = []
    masks = []
    total_reward = 0
    done = False
    while not done:
        action, log_prob = select_action(state, model)
        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)
        log_probs.append(log_prob)
        total_reward += reward
        
        next_state_tensor = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0)
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        
        _, value = model(state_tensor)
        values.append(value)
        
        masks.append(1 - done)
        
        state = next_state
    
    returns = []
    R = 0
    for r in rewards[::-1]:
        R = r + gamma * R
        returns.insert(0, R)
    returns = torch.tensor(returns)
    
    log_probs = torch.stack(log_probs)
    values = torch.stack(values).squeeze(1)
    masks = torch.tensor(masks, dtype=torch.float32)
    
    advantages = returns - values
    critic_loss = advantages.pow(2).mean()
    
    ratio = torch.exp(log_probs - log_probs.detach())
    surr1 = ratio * advantages
    surr2 = torch.clamp(ratio, 1.0 - epsilon, 1.0 + epsilon) * advantages
    actor_loss = -torch.min(surr1, surr2).mean()
    
    loss = actor_loss + critic_loss
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f'Episodio {episode + 1}, Recompensa total: {total_reward}')

env.close()


**A3C (Asynchronous Advantage Actor-Critic)**:

A3C es un algoritmo de aprendizaje asincrónico donde múltiples agentes independientes aprenden simultáneamente y actualizan una política global. Utiliza la misma estructura que el actor-critic, pero distribuye el entrenamiento en múltiples instancias del entorno.

En resumen, los métodos de aprendizaje por refuerzo abarcan una variedad de enfoques, cada uno con sus propias fortalezas y desafíos. Desde los métodos basados en el valor como Q-Learning y SARSA, hasta los métodos basados en la política como REINFORCE y Actor-Critic, y finalmente los métodos híbridos como DDPG, PPO y A3C, todos juegan un papel crucial en el desarrollo de agentes inteligentes capaces de tomar decisiones en entornos complejos y dinámicos.

### Métodos de Monte Carlo: simulación y estimación de valores*

Los métodos de Monte Carlo son una clase de algoritmos computacionales que dependen de muestreos aleatorios repetidos para obtener resultados numéricos. Se utilizan principalmente en optimización, integración numérica y generación de muestras de una distribución de probabilidad. En el contexto del aprendizaje por refuerzo, los métodos de Monte Carlo se usan para estimar el valor esperado de una política dada, promediando las recompensas observadas a lo largo de múltiples episodios.

**Simulación y estimación de valores con Monte Carlo**

En el aprendizaje por refuerzo, la simulación mediante métodos de Monte Carlo implica ejecutar varios episodios de interacción entre el agente y el entorno para recopilar datos sobre las recompensas obtenidas. A partir de estas simulaciones, se pueden calcular las estimaciones de valor. Estas estimaciones se basan en la premisa de que, a largo plazo, las recompensas acumuladas reflejan el valor esperado de un estado o una acción.

Para estimar los valores de una política, se sigue un procedimiento que incluye:

1. **Generación de episodios:** Se generan múltiples episodios siguiendo la política actual, donde un episodio es una secuencia de estados, acciones y recompensas que termina en un estado terminal.
2. **Cálculo de retornos:** Para cada estado visitado en un episodio, se calcula el retorno, que es la suma de recompensas futuras descontadas.
3. **Promedio de retornos:** Se promedian los retornos de todos los episodios en los que se visitó un estado específico para obtener una estimación de su valor.

Una característica fundamental de los métodos de Monte Carlo es que requieren episodios completos, lo que significa que sólo se actualizan los valores al final de un episodio. Esto puede ser una limitación en entornos donde los episodios son largos o no terminan.

**Métodos de diferencias temporales (TD): TD(λ) y n-step TD**

Los métodos de diferencias temporales combinan las ventajas del aprendizaje Monte Carlo y el aprendizaje dinámico, actualizando las estimaciones de valores en función de las diferencias temporales, es decir, la diferencia entre las estimaciones de valor consecutivas.

**TD(λ)**

TD(λ) es una técnica que unifica los métodos TD y Monte Carlo mediante el uso de trazas de elegibilidad, que son variables que asignan crédito a los estados y acciones visitados recientemente. λ es un parámetro que controla la ponderación de las actualizaciones.

1. **Trazas de elegibilidad:** Se utilizan para dar crédito a los estados visitados recientemente. A medida que el agente se mueve a través del entorno, las trazas de elegibilidad se actualizan, decayendo con cada paso.
2. **Actualización de valores:** Las actualizaciones de valores se realizan no solo en función del estado actual y el siguiente estado, sino también en función de los estados anteriores, ponderados por sus trazas de elegibilidad.

La fórmula general de actualización para TD(λ) es:

$$V(s) \leftarrow V(s) + \alpha \delta_t e_t(s)$$
donde:
- $\alpha$ es la tasa de aprendizaje,
- $\delta_t$ es el error de TD,
- $e_t(s)$ es la traza de elegibilidad.

**n-step TD**

El método n-step TD extiende la idea básica de TD al utilizar recompensas de los siguientes n pasos en lugar de solo la recompensa inmediata y el valor del siguiente estado. Este método actualiza los valores basándose en n pasos futuros de interacción con el entorno.

1. **Recompensas acumuladas:** Para cada estado, se acumulan las recompensas de los próximos n pasos.
2. **Actualización de valores:** Los valores se actualizan utilizando esta suma de recompensas, proporcionando un equilibrio entre la actualización a corto plazo (TD(0)) y la actualización a largo plazo (Monte Carlo).

**Exploración vs. explotación: estrategias epsilon-greedy, Upper Confidence Bound (UCB)**

En el aprendizaje por refuerzo, un desafío clave es balancear la exploración de nuevas acciones con la explotación de las acciones conocidas que proporcionan las mayores recompensas. Este dilema se conoce como el trade-off exploración-explotación.

**Estrategia epsilon-greedy**

La estrategia epsilon-greedy es una de las técnicas más simples y efectivas para gestionar este trade-off. En esta estrategia:

1. **Exploración:** Con una probabilidad \(\epsilon\), el agente selecciona una acción al azar, lo que permite explorar nuevas acciones.
2. **Explotación:** Con una probabilidad \(1 - \epsilon\), el agente selecciona la acción con el mayor valor esperado (explota el conocimiento actual).

El parámetro \(\epsilon\) se puede ajustar durante el entrenamiento, a menudo comenzando con un valor alto para fomentar la exploración y disminuyéndolo gradualmente para favorecer la explotación a medida que el agente aprende más sobre el entorno.

**Upper Confidence Bound (UCB)**

El método Upper Confidence Bound es otra estrategia para balancear exploración y explotación, utilizando una forma más teórica y matemática basada en la teoría de la toma de decisiones en condiciones de incertidumbre.

1. **Valor de confianza:** UCB asigna a cada acción un valor de confianza que aumenta con la incertidumbre de la estimación del valor de esa acción.
2. **Selección de acciones:** El agente selecciona la acción con el valor de confianza más alto, lo que favorece las acciones con altos valores esperados y aquellas que han sido menos exploradas.

La fórmula general para el valor de confianza en UCB es:
$$Q(a) + c \sqrt{\frac{\ln(t)}{N(a)}}$$
donde:
- $Q(a)$ es el valor estimado de la acción $a$,
- $c$ es un parámetro que controla el grado de exploración,
- $t$ es el número total de selecciones de acciones,
- $N(a)$ es el número de veces que la acción $a$ ha sido seleccionada.

UCB proporciona un marco matemáticamente riguroso para el trade-off exploración-explotación, asegurando que cada acción sea seleccionada un número suficiente de veces para obtener estimaciones precisas de su valor.


Presentamos implementaciones de las estrategia Upper Confidence Bound (UCB) en PyTorch para el problema del bandido multi-brazo. La estrategia UCB es útil en situaciones donde se quiere balancear la exploración y la explotación sin necesidad de un parámetro de exploración explícito como epsilon.

**Implementación de UCB1**

El algoritmo UCB1 selecciona la acción que maximiza el valor esperado más un término de exploración que disminuye a medida que se incrementa el número de veces que se ha seleccionado esa acción.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym
import math

class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

def select_action(state, q_network, counts, total_counts, epsilon=1e-5):
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        q_values = q_network(state).numpy().flatten()
    
    adjusted_counts = np.maximum(counts, epsilon)
    adjusted_total_counts = max(total_counts, epsilon)
    ucb_values = q_values + np.sqrt((2 * np.log(adjusted_total_counts)) / adjusted_counts)
    action = np.argmax(ucb_values)
    
    return action

env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
lr = 0.001
gamma = 0.99
episodes = 500

q_network = QNetwork(state_dim, action_dim)
optimizer = optim.Adam(q_network.parameters(), lr=lr)
criterion = nn.MSELoss()

counts = np.zeros(action_dim)
total_counts = 0

for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False
    while not done:
        action = select_action(state, q_network, counts, total_counts)
        counts[action] += 1
        total_counts += 1

        next_state, reward, done, _ = env.step(action)
        total_reward += reward

        next_q_values = q_network(torch.tensor(next_state, dtype=torch.float32).unsqueeze(0))
        max_next_q_value = next_q_values.max().item()
        target = reward + (gamma * max_next_q_value * (1 - done))
        
        q_values = q_network(torch.tensor(state, dtype=torch.float32).unsqueeze(0))
        target_f = q_values.clone().detach()
        target_f[0][action] = target

        loss = criterion(q_values, target_f)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        state = next_state

    print(f'Episodio {episode + 1}, Recompensa total: {total_reward}')

env.close()


UCB-V es una variante del algoritmo UCB que considera la varianza en la estimación de las recompensas.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym
import math

class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

def select_action(state, q_network, counts, total_counts, q_variances, epsilon=1e-5):
    state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
        q_values = q_network(state).numpy().flatten()
    
    adjusted_counts = counts + epsilon
    ucb_values = q_values + np.sqrt((2 * np.log(total_counts + epsilon) * q_variances) / adjusted_counts)
    action = np.argmax(ucb_values)
    
    return action

env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
lr = 0.001
gamma = 0.99
episodes = 500

q_network = QNetwork(state_dim, action_dim)
optimizer = optim.Adam(q_network.parameters(), lr=lr)
criterion = nn.MSELoss()

counts = np.zeros(action_dim)
q_variances = np.ones(action_dim)  # Initial variances can be set to 1
total_counts = 0

for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False
    episode_rewards = []
    while not done:
        action = select_action(state, q_network, counts, total_counts, q_variances)
        counts[action] += 1
        total_counts += 1

        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        episode_rewards.append(reward)

        next_q_values = q_network(torch.tensor(next_state, dtype=torch.float32).unsqueeze(0))
        max_next_q_value = next_q_values.max().item()
        target = reward + (gamma * max_next_q_value * (1 - done))
        
        q_values = q_network(torch.tensor(state, dtype=torch.float32).unsqueeze(0))
        target_f = q_values.clone().detach()
        target_f[0][action] = target

        loss = criterion(q_values, target_f)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        state = next_state

    # Update Q variances after each episode
    episode_mean_reward = np.mean(episode_rewards)
    for action in range(action_dim):
        action_rewards = [r for i, r in enumerate(episode_rewards) if i % action_dim == action]
        if len(action_rewards) > 1:
            q_variances[action] = np.var(action_rewards)
    
    print(f'Episodio {episode + 1}, Recompensa total: {total_reward}')

env.close()


### Deep Q-Networks (DQN)

Deep Q-Networks (DQN) representan una evolución significativa en el campo del Aprendizaje por Refuerzo (RL), combinando técnicas tradicionales de RL con redes neuronales profundas. Este enfoque fue popularizado por la investigación de DeepMind, donde se demostró que una red neuronal podía aprender a jugar videojuegos de Atari a nivel humano.

En los métodos tradicionales de RL, la función Q se representa mediante tablas (tabular methods) que mapean cada par estado-acción a un valor Q. Sin embargo, este enfoque no es escalable a entornos con grandes espacios de estados y acciones. Aquí es donde entran las redes neuronales: en lugar de almacenar valores Q explícitamente, una red neuronal se entrena para aproximar la función Q.

La arquitectura de un DQN es bastante simple: se utiliza una red neuronal con varias capas ocultas que toma el estado del entorno como entrada y produce un valor Q para cada posible acción. La actualización de los pesos de la red neuronal se realiza mediante un proceso de backpropagation, donde el objetivo es minimizar el error entre el valor Q estimado y el valor Q objetivo, el cual se define mediante la ecuación de Bellman.

Veamos una simple implementación en Pytorch.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym

class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
lr = 0.001
gamma = 0.99
episodes = 500
epsilon = 0.1

dqn = DQN(state_dim, action_dim)
optimizer = optim.Adam(dqn.parameters(), lr=lr)
criterion = nn.MSELoss()

for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    done = False
    while not done:
        if np.random.rand() < epsilon:
            action = env.action_space.sample()
        else:
            state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
            q_values = dqn(state_tensor)
            action = torch.argmax(q_values).item()
        
        next_state, reward, done, _ = env.step(action)
        total_reward += reward

        next_state_tensor = torch.tensor(next_state, dtype=torch.float32).unsqueeze(0)
        target = reward + gamma * torch.max(dqn(next_state_tensor)).item() * (1 - done)
        
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        q_values = dqn(state_tensor)
        target_f = q_values.clone().detach()
        target_f[0][action] = target

        loss = criterion(q_values, target_f)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        state = next_state

    print(f'Episodio {episode + 1}, Recompensa total: {total_reward}')

env.close()


### Redes Actor-Critic
El enfoque Actor-Critic combina las ventajas de los métodos basados en políticas y los basados en valor. En lugar de tener una única red que aprenda la política o el valor Q, se utilizan dos redes: una para la política (actor) y otra para el valor (critic).

La red Actor se encarga de seleccionar acciones según una política aprendida, mientras que la red Critic evalúa estas acciones proporcionando una estimación del valor Q. 

El Actor mejora su política utilizando las críticas del Critic, haciendo que este enfoque sea más estable y eficiente en comparación con los métodos puramente basados en políticas.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym
from torch.utils.data import BatchSampler, SubsetRandomSampler

torch.autograd.set_detect_anomaly(True)

class ActorCriticNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(ActorCriticNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.actor = nn.Linear(128, action_dim)
        self.critic = nn.Linear(128, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        action_probs = torch.softmax(self.actor(x), dim=-1)
        state_values = self.critic(x)
        return action_probs, state_values

def compute_gae(rewards, masks, values, gamma=0.99, tau=0.95):
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
lr = 0.0003
gamma = 0.99
tau = 0.95
episodes = 500
clip_param = 0.2
ppo_epochs = 10
batch_size = 64

model = ActorCriticNetwork(state_dim, action_dim)
optimizer = optim.Adam(model.parameters(), lr=lr)

for episode in range(episodes):
    state = env.reset()
    log_probs = []
    values = []
    rewards = []
    masks = []
    entropy = 0
    total_reward = 0
    done = False
    states = []
    actions = []

    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        dist, value = model(state_tensor)
        action = torch.multinomial(dist, 1).item()

        next_state, reward, done, _ = env.step(action)
        total_reward += reward

        log_prob = torch.log(dist.squeeze(0)[action])
        entropy += -torch.sum(dist * torch.log(dist), dim=-1).mean()

        log_probs.append(log_prob)
        values.append(value)
        rewards.append(reward)
        masks.append(1 - done)

        states.append(state)
        actions.append(action)
        state = next_state

    _, next_value = model(torch.tensor(next_state, dtype=torch.float32).unsqueeze(0))
    values.append(next_value)
    returns = compute_gae(rewards, masks, values, gamma, tau)

    log_probs = torch.stack(log_probs)
    returns = torch.stack(returns).detach()
    values = torch.stack(values)[:-1].detach()
    states = torch.tensor(np.array(states), dtype=torch.float32)
    actions = torch.tensor(actions, dtype=torch.int64)

    advantages = returns - values

    for _ in range(ppo_epochs):
        sampler = BatchSampler(SubsetRandomSampler(range(len(rewards))), batch_size, False)
        for indices in sampler:
            sampled_states = states[indices]
            sampled_actions = actions[indices]
            sampled_returns = returns[indices]
            sampled_advantages = advantages[indices]
            sampled_log_probs = log_probs[indices]

            dist, value = model(sampled_states)
            new_log_probs = torch.log(dist.gather(1, sampled_actions.unsqueeze(-1)).squeeze(-1))
            ratio = (new_log_probs - sampled_log_probs).exp()
            surr1 = ratio * sampled_advantages
            surr2 = torch.clamp(ratio, 1.0 - clip_param, 1.0 + clip_param) * sampled_advantages
            actor_loss = -torch.min(surr1, surr2).mean()
            critic_loss = (sampled_returns - value).pow(2).mean()
            loss = 0.5 * critic_loss + actor_loss - 0.001 * entropy

            optimizer.zero_grad()
            # Retain graph on the last PPO epoch
            retain_graph = _ < ppo_epochs - 1
            loss.backward(retain_graph=retain_graph)
            optimizer.step()

    print(f'Episodio {episode + 1}, Recompensa total: {total_reward}')

env.close()


### Algoritmos avanzados

**Proximal Policy Optimization (PPO)**

PPO es uno de los algoritmos más robustos y populares en el aprendizaje por refuerzo. La idea principal detrás de PPO es limitar el cambio en la política en cada actualización para mantener la estabilidad y eficiencia del entrenamiento. PPO emplea una técnica llamada "clipping" para evitar cambios demasiado grandes en la política.

#### Asynchronous Advantage Actor-Critic (A3C)

A3C es otro algoritmo avanzado que utiliza múltiples agentes (hilos) que interactúan con el entorno de forma paralela. Cada agente tiene su propia copia de la red neuronal y actualiza los parámetros de la red global de forma asíncrona. Este enfoque mejora la eficiencia del entrenamiento y la estabilidad del modelo.



Estos enfoques avanzados han demostrado ser altamente efectivos para resolver problemas complejos en el campo del aprendizaje por refuerzo, proporcionando un marco robusto y eficiente para la toma de decisiones autónoma en una variedad de entornos.

### Integración de aprendizaje por refuerzo y transformers
La integración de los Transformers, un tipo de modelo de aprendizaje profundo que ha revolucionado el procesamiento del lenguaje natural (NLP), con el aprendizaje por refuerzo (RL) ofrece nuevas oportunidades para mejorar el rendimiento y la eficiencia de los agentes inteligentes. Los Transformers, originalmente diseñados para tareas de NLP, han demostrado ser extremadamente poderosos en capturar dependencias a largo plazo y modelar secuencias complejas. Estos atributos los hacen adecuados para diversas aplicaciones de RL, donde la secuenciación y el modelado de decisiones a largo plazo son cruciales.

**Transformers para RL**

El uso de transformers en RL se puede dividir en dos áreas principales: modelar políticas y modelar funciones de valor. En ambas áreas, los Transformers pueden aprovechar su capacidad para manejar secuencias y capturar relaciones a largo plazo para mejorar el rendimiento de los agentes de RL.


En el contexto del RL, una política define el comportamiento de un agente, mapeando estados del entorno a acciones. Tradicionalmente, las políticas se han modelado usando redes neuronales recurrentes (RNNs) o redes neuronales convolucionales (CNNs). Sin embargo, los Transformers pueden ofrecer ventajas significativas debido a su capacidad para procesar secuencias completas y capturar dependencias complejas entre las observaciones del entorno.

Un transformer puede ser entrenado para modelar una política de RL mediante el aprendizaje supervisado, donde se le proporciona una secuencia de estados y acciones óptimas. El Transformer aprende a predecir la siguiente acción dada la secuencia actual de estados. Una vez entrenado, el Transformer puede ser usado para inferir la acción óptima en cada paso del tiempo durante la interacción del agente con el entorno.

**Uso de transformers para modelar funciones de valor**

Las funciones de valor en RL evalúan la calidad de los estados o las acciones, proporcionando una estimación de las recompensas futuras esperadas. Los Transformers pueden ser utilizados para aproximar estas funciones de valor mediante la integración de la información de secuencias completas de estados y recompensas.

La arquitectura de un transformer, con su mecanismo de atención, puede capturar las relaciones entre estados a lo largo de una secuencia, permitiendo una estimación más precisa de las recompensas futuras. Esto es particularmente útil en entornos donde las decisiones a largo plazo tienen un impacto significativo en las recompensas.

Para ilustrar el uso de transformers en RL, consideremos una implementación simplificada donde un transformer se utiliza para modelar una política en el entorno de CartPole de OpenAI Gym.

Primero, definimos el transformer que será utilizado para modelar la política. El transformer recibe una secuencia de estados como entrada y produce una distribución de probabilidad sobre las acciones posibles.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym

class TransformerPolicy(nn.Module):
    def __init__(self, state_dim, action_dim, d_model=128, nhead=8, num_layers=2):
        super(TransformerPolicy, self).__init__()
        self.embedding = nn.Linear(state_dim, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.fc = nn.Linear(d_model, action_dim)
    
    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # Permute to [sequence, batch, features]
        transformer_output = self.transformer(x, x)
        output = self.fc(transformer_output)
        return torch.softmax(output, dim=-1)

env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
policy_net = TransformerPolicy(state_dim, action_dim)
optimizer = optim.Adam(policy_net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for episode in range(500):
    state = env.reset()
    total_reward = 0
    done = False
    states = []
    actions = []
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        states.append(state_tensor)
        state_sequence = torch.cat(states).unsqueeze(1)  # Add batch dimension
        action_probs = policy_net(state_sequence)
        action = torch.multinomial(action_probs[-1, -1], 1).item()
        actions.append(torch.tensor([action], dtype=torch.int64))
        
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        state = next_state

        if done:
            state_sequence = torch.cat(states).unsqueeze(1)  # Add batch dimension
            action_sequence = torch.cat(actions).unsqueeze(1)
            optimizer.zero_grad()
            outputs = policy_net(state_sequence)
            outputs = outputs.view(-1, action_dim)  # Flatten the outputs for CrossEntropyLoss
            loss = criterion(outputs, action_sequence.view(-1))
            loss.backward()
            optimizer.step()
            
    print(f'Episodio {episode + 1}, Recompensa total: {total_reward}')

env.close()


Para modelar funciones de valor con transformers, el proceso es similar. Sin embargo, en lugar de predecir acciones, el transformer predice el valor esperado de los estados o las acciones. 

A continuación se muestra un ejemplo de cómo un transformer puede ser utilizado para aproximar la función de valor en el entorno de CartPole.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym

class TransformerValue(nn.Module):
    def __init__(self, state_dim, d_model=128, nhead=8, num_layers=2):
        super(TransformerValue, self).__init__()
        self.embedding = nn.Linear(state_dim, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.fc = nn.Linear(d_model, 1)
    
    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # Permute to [sequence, batch, features]
        transformer_output = self.transformer(x, x)
        output = self.fc(transformer_output).squeeze(1)
        return output

env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
value_net = TransformerValue(state_dim)
optimizer = optim.Adam(value_net.parameters(), lr=0.001)
criterion = nn.MSELoss()

for episode in range(500):
    state = env.reset()
    total_reward = 0
    done = False
    states = []
    rewards = []
    while not done:
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        states.append(state_tensor)
        
        action = env.action_space.sample()
        next_state, reward, done, _ = env.step(action)
        rewards.append(reward)
        total_reward += reward
        state = next_state

        if done:
            state_sequence = torch.cat(states).unsqueeze(1)  # Add batch dimension
            state_sequence = state_sequence.permute(1, 0, 2)  # [sequence, batch, features]
            returns = [sum(rewards[i:]) for i in range(len(rewards))]
            returns = torch.tensor(returns, dtype=torch.float32).unsqueeze(1)
            
            optimizer.zero_grad()
            values = value_net(state_sequence)
            loss = criterion(values, returns)
            loss.backward()
            optimizer.step()
            
    print(f'Episodio {episode + 1}, Recompensa total: {total_reward}')

env.close()


La integración de transformers, como BERT, GPT-3, GPT-4, y GPT-5, con el aprendizaje por refuerzo (RL) representa un avance significativo en la creación de agentes inteligentes más eficientes y versátiles. Estos modelos, que han demostrado un rendimiento superior en tareas de procesamiento del lenguaje natural (NLP), también tienen el potencial de mejorar los sistemas de RL al aprovechar sus capacidades para modelar secuencias y capturar dependencias a largo plazo.


Los transformers se pueden utilizar para modelar políticas en RL, aprovechando su capacidad para procesar secuencias completas de estados y acciones. En lugar de depender únicamente de las observaciones actuales, un Transformer puede tener en cuenta una historia más extensa de estados para decidir la mejor acción. Esto es particularmente útil en entornos donde la información relevante se distribuye a lo largo del tiempo.

Por ejemplo, en un entorno de juego, un transformer puede utilizar la historia completa de movimientos para decidir la mejor acción actual. La arquitectura del Transformer permite manejar esta secuencia de forma eficiente, capturando relaciones complejas y dependencias a largo plazo.


En general el uso de transformers en RL ofrece varias ventajas potenciales:

* Captura de dependencias a largo plazo: Los Transformers son capaces de capturar dependencias a largo plazo en las secuencias de estados y acciones, lo cual es crucial en tareas de RL donde las decisiones a largo plazo impactan significativamente las recompensas.

* Escalabilidad: Los Transformers pueden manejar grandes secuencias de datos y múltiples tipos de entradas, lo que los hace adecuados para entornos complejos y de alta dimensionalidad.

* Flexibilidad: Los Transformers pueden ser fácilmente adaptados para modelar políticas, funciones de valor y otros componentes críticos en los algoritmos de RL, proporcionando un marco unificado y flexible.


### Ejercicios

1 . Implementa un entorno simple, como el problema del camino más corto en un laberinto.

- Define los elementos básicos: agente, entorno, estados, acciones y recompensas.
- Describe cómo el agente interactúa con el entorno y cómo se calculan las recompensas.

2 . Procesos de decisión de Markov (MDPs)

- Modela un MDP para un problema de control de inventario.
- Define los estados, acciones, probabilidades de transición y recompensas.
- Escribe una política simple ($\pi$) que el agente puede seguir.

3 . Implementación de Q-Learning

- Implementa el algoritmo Q-Learning para resolver el problema del laberinto del Ejercicio 1.
- Ajusta los parámetros de aprendizaje y observa cómo cambian las políticas óptimas.

4 .Algoritmo REINFORCE

- Implementa el algoritmo REINFORCE en un entorno de bandido multi-brazo.
- Analiza cómo varía el rendimiento del agente con diferentes tasas de aprendizaje y estrategias de exploración.

5 . Actor-Critic

- Implementa un algoritmo Actor-Critic para un entorno de control continuo, como el problema del equilibrio de un péndulo.
- Compara el rendimiento con otros métodos basados en políticas y basados en valores.

6 . Monte Carlo y diferencias temporales (TD)

- Implementa un método de Monte Carlo para estimar los valores de estado en un problema de caminata aleatoria.
- Implementa el método TD(λ) y compara los resultados con los obtenidos por Monte Carlo.

7 . Exploración vs. Explotación

- Implementa y compara las estrategias epsilon-greedy y Upper Confidence Bound (UCB) en un problema de bandido multi-brazo.
- Analiza cómo cada estrategia afecta la exploración y explotación.

8. Deep Q-Networks (DQN)

- Implementa un DQN para resolver el problema del Cart-Pole.
- Analiza cómo el uso de redes neuronales mejora la aproximación de la función Q comparado con métodos tabulares.

9 . Redes Actor-Critic

- Implementa una red Actor-Critic usando una arquitectura de red neuronal profunda.
- Aplica este modelo a un entorno más complejo, como el entorno de Ataris en OpenAI Gym.

10 . PPO y A3C

- Implementa los algoritmos PPO y A3C para un entorno de control continuo.
- Compara el rendimiento y la estabilidad de ambos algoritmos en el entorno seleccionado.

11 .Transformers para RL

- Investiga y resume cómo los Transformers se pueden utilizar para modelar políticas y funciones de valor en entornos de RL.
- Implementa una política basada en Transformers para un problema de generación de lenguaje natural (NLG), como la creación de diálogos en un chatbot.
12. Transformers en modelos de grandes lenguajes (LLMs)

- Utiliza un modelo de Transformer preentrenado (como GPT-3) y adapta su arquitectura para un problema de RL, como la generación de respuestas en un diálogo interactivo.
- Experimenta con diferentes enfoques de fine-tuning para mejorar la coherencia y relevancia de las respuestas generadas.


In [ ]:
## Tus respuestas